In [4]:
# daily_update.py
import pandas as pd
import numpy as np
import mysql.connector
from faker import Faker
from datetime import datetime, timedelta
import random
import os
import time

# 设置随机种子
random.seed(42)
np.random.seed(42)

# 设置 Faker
fake = Faker('zh_CN')
fake_en = Faker()

# 数据库配置
DB_CONFIG = {
    'host': '127.0.0.1',
    'user': 'root',
    'password': 'Taylor@1989',  # 使用您提供的密码
    'database': 'employee_db',
    'port': 3306
}

# 常量设置
TOTAL_EMPLOYEES = 500
MONTHLY_LEAVERS = (18, 25)
MONTHLY_HIRING = (18, 20)
HIRING_START_DAYS = (30, 60)

DEPARTMENTS = {
    'Sales': 0.30,
    'Marketing': 0.20,
    'Engineering': 0.25,
    'HR': 0.10,
    'Legal': 0.05,
    'Operations': 0.10
}

SALARY_LEVELS = {
    'low': 0.50,
    'medium': 0.40,
    'high': 0.10
}

SALARY_RANGES = {
    'Sales': {'min': 150, 'max': 500},
    'Marketing': {'min': 120, 'max': 450},
    'Engineering': {'min': 180, 'max': 600},
    'HR': {'min': 100, 'max': 400},
    'Legal': {'min': 150, 'max': 500},
    'Operations': {'min': 80, 'max': 350}
}

# 依赖函数（从 Documents/turnover tracking/daily_update.ipynb复制）
def generate_employee_ids(count):
    return random.sample(range(1000, 100000), count)

def generate_satisfaction_score(is_leaving):
    if is_leaving:
        return round(max(0, min(1, np.random.beta(2, 3) * 0.8)), 2)
    return round(max(0, min(1, np.random.beta(4, 2) * 0.9 + 0.1)), 2)

def generate_evaluation_score():
    return round(random.uniform(0.4, 0.6) if random.random() < 0.5 else random.uniform(0.8, 1.0), 2)

def generate_project_count(is_leaving):
    if is_leaving and random.random() < 0.4:
        return random.randint(6, 7)
    return max(0, min(7, int(np.random.normal(3.8, 1.5))))

def generate_monthly_hours(is_leaving):
    if is_leaving and random.random() < 0.6:
        return random.randint(250, 310)
    return random.randint(96, 150) if random.random() < 0.5 else random.randint(250, 280)

def generate_years_at_company(is_leaving):
    if is_leaving and random.random() < 0.6:
        return random.randint(3, 5)
    return max(0, min(10, int(np.random.gamma(2, 1.8))))

def generate_hire_date(years_at_company):
    current_date = datetime.now()
    years_ago = current_date - timedelta(days=years_at_company * 365 + random.randint(-180, 180))
    return years_ago.strftime('%Y-%m-%d')

def generate_termination_date(is_leaving):
    if is_leaving:
        current_date = datetime.now()
        recent_date = current_date - timedelta(days=random.randint(0, 30))
        return recent_date.strftime('%Y-%m-%d')
    return None

def generate_work_accident(is_leaving):
    return 1 if random.random() < (0.05 if is_leaving else 0.18) else 0

def generate_promotion(is_leaving):
    return 1 if random.random() < (0.005 if is_leaving else 0.03) else 0

def generate_department():
    return random.choices(list(DEPARTMENTS.keys()), weights=list(DEPARTMENTS.values()), k=1)[0]

def generate_salary_level():
    return random.choices(list(SALARY_LEVELS.keys()), weights=list(SALARY_LEVELS.values()), k=1)[0]

def generate_actual_salary(department):
    salary_range = SALARY_RANGES[department]
    return random.randint(salary_range['min'], salary_range['max']) * 1000

def get_start_date():
    """从 MySQL 获取 start_date"""
    try:
        conn = mysql.connector.connect(**DB_CONFIG)
        cursor = conn.cursor()
        cursor.execute("CREATE TABLE IF NOT EXISTS metadata (meta_key VARCHAR(50), meta_value VARCHAR(50))")
        cursor.execute("SELECT meta_value FROM metadata WHERE meta_key = 'start_date'")
        result = cursor.fetchone()
        if result:
            start_date = datetime.strptime(result[0], '%Y-%m-%d %H:%M:%S')
        else:
            start_date = datetime.now()
            cursor.execute("INSERT INTO metadata (meta_key, meta_value) VALUES ('start_date', %s)", (start_date.strftime('%Y-%m-%d %H:%M:%S'),))
            conn.commit()
        cursor.close()
        conn.close()
        return start_date
    except mysql.connector.Error as e:
        print(f"获取start_date失败: {e}")
        return datetime.now()

def daily_update(start_date=None):
    current_date = datetime.now()
    current_date_str = current_date.strftime('%Y-%m-%d %H:%M:%S')
    print(f"开始每日更新: {current_date_str}")

    days_since_start = 0
    if start_date:
        days_since_start = (current_date - start_date).days

    try:
        conn = mysql.connector.connect(**DB_CONFIG)
        cursor = conn.cursor(dictionary=True)

        cursor.execute("SELECT COUNT(*) as total FROM employees WHERE turnover = 0")
        current_count = cursor.fetchone()['total']
        print(f"当前在职员工数: {current_count}")

        monthly_leavers = random.randint(MONTHLY_LEAVERS[0], MONTHLY_LEAVERS[1])
        daily_leavers = max(1, round(monthly_leavers / 30))
        monthly_hiring = 0
        daily_hiring = 0

        if days_since_start >= random.randint(HIRING_START_DAYS[0], HIRING_START_DAYS[1]):
            monthly_hiring = random.randint(MONTHLY_HIRING[0], MONTHLY_HIRING[1])
            daily_hiring = max(1, round(monthly_hiring / 30))

        print(f"本次更新: {daily_leavers}人离职, {daily_hiring}人新入职")

        cursor.execute("SELECT employee_id FROM employees WHERE turnover = 0 ORDER BY RAND() LIMIT %s", (daily_leavers,))
        leaving_employees = cursor.fetchall()

        for emp in leaving_employees:
            cursor.execute(
                "UPDATE employees SET turnover = 1, satisfaction = %s, termination_date = %s, last_updated = %s WHERE employee_id = %s",
                (round(random.uniform(0.1, 0.5), 2), current_date.strftime('%Y-%m-%d'), current_date_str, emp['employee_id'])
            )

        cursor.execute("SELECT MAX(employee_id) as max_id FROM employees")
        max_id = cursor.fetchone()['max_id'] or 0
        new_employees = []

        for i in range(daily_hiring):
            department = generate_department()
            new_id = max_id + i + 1
            new_employees.append({
                'employee_id': new_id,
                'name': fake_en.name(),
                'department': department,
                'salary_level': generate_salary_level(),
                'actual_salary': generate_actual_salary(department),
                'turnover': 0,
                'satisfaction': round(random.uniform(0.6, 0.95), 2),
                'evaluation': round(random.uniform(0.7, 0.9), 2),
                'project_count': random.randint(1, 3),
                'average_monthly_hours': random.randint(160, 200),
                'years_at_company': 0,
                'hire_date': current_date.strftime('%Y-%m-%d'),
                'termination_date': None,
                'work_accident': 0,
                'promotion': 0,
                'last_updated': current_date_str
            })

        for emp in new_employees:
            cursor.execute("""
            INSERT INTO employees (
                employee_id, name, department, salary_level, actual_salary, 
                turnover, satisfaction, evaluation, project_count, 
                average_monthly_hours, years_at_company, hire_date, 
                termination_date, work_accident, promotion, last_updated
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """, (
                emp['employee_id'], emp['name'], emp['department'], emp['salary_level'], emp['actual_salary'],
                emp['turnover'], emp['satisfaction'], emp['evaluation'], emp['project_count'],
                emp['average_monthly_hours'], emp['years_at_company'], emp['hire_date'],
                emp['termination_date'], emp['work_accident'], emp['promotion'], emp['last_updated']
            ))

        conn.commit()

        cursor.execute("SELECT COUNT(*) as total FROM employees WHERE turnover = 0")
        updated_count = cursor.fetchone()['total']
        cursor.execute("SELECT COUNT(*) as total FROM employees WHERE turnover = 1")
        leaving_count = cursor.fetchone()['total']
        print(f"更新完成: 在职员工 {updated_count} 人，历史离职员工 {leaving_count} 人")
        print(f"当前离职率: {leaving_count/(updated_count+leaving_count):.2%}")

        cursor.execute("SELECT * FROM employees")
        all_employees = cursor.fetchall()
        df = pd.DataFrame(all_employees)
        csv_filename = f"employee_data_{current_date.strftime('%Y%m%d')}.csv"
        df.to_csv(csv_filename, index=False, encoding='utf-8-sig')
        print(f"数据已保存到 {csv_filename}")

        cursor.close()
        conn.close()
    except mysql.connector.Error as e:
        print(f"数据库更新失败: {e}")
        print("请确认MySQL连接信息是否正确")
        with open('error.log', 'a') as f:
            f.write(f"{datetime.now()}: 数据库更新失败: {e}\n")
    except Exception as e:
        print(f"更新过程发生错误: {e}")
        with open('error.log', 'a') as f:
            f.write(f"{datetime.now()}: 更新过程发生错误: {e}\n")

    print(f"每日更新完成: {current_date_str}")

if __name__ == "__main__":
    start_date = get_start_date()
    daily_update(start_date)

开始每日更新: 2025-05-02 14:54:58
当前在职员工数: 480
本次更新: 1人离职, 0人新入职
更新完成: 在职员工 479 人，历史离职员工 21 人
当前离职率: 4.20%
数据已保存到 employee_data_20250502.csv
每日更新完成: 2025-05-02 14:54:58
